In [1]:
import pandas as pd
import numpy as np

In [2]:
%sh ls '/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_09_23_34_25/'

benchmark|PROD|CALTECH101|VGG16|6982|1695|SINGLE_INSTANCE|p2.8xlarge|p2.xlarge|1|NA|1|40|0.001|32.h5
benchmark|PROD|CALTECH101|VGG16|6982|1695|SINGLE_INSTANCE|p2.8xlarge|p2.xlarge|1|NA|1|40|0.001|32.log

In [3]:
batch_32_result_mapping = {"VGG16":
                                {"single": '/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_09_23_34_25/',
                                  "-818": '/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_09_22_24_36/',
                                  "16": '/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_11_00_29_30/', # With load model from S3
                                 # Another "16" with normal load model: 2019_11_09_23_33_45
                                  "32": '/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_10_16_23_01/'},
                           "INCEPTION3":
                              {"single": "/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_10_00_19_28/",
                                "-818": "/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_10_00_51_12/",
                                "16": "/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_09_23_38_02/",
                                "32": "/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_10_16_34_57/"},
                         }

In [4]:
import glob
import os
import re
from datetime import datetime

def extract_result_from_log(log_filepath):
  res = []
  
  filename = os.path.basename(log_filepath)
  _, dataset, env, model, train_shape, test_shape, instance, driver_type, worker_type, num_workers, np, repeat, epochs, learning_rate, batch_size = filename.split("|")
  batch_size = batch_size.split(".")[0]
  
  current_repeat_ord, last_epoch, last_epoch_time, current_epoch_time, epoch_run_time = None, None, None, None, None
  
  def append_res():
    res.append([last_epoch_time, dataset, model, train_shape, test_shape, instance, driver_type, worker_type, num_workers, np, repeat, epochs, 
                learning_rate, batch_size, current_repeat_ord, last_epoch, epoch_run_time, current_epoch_time])
    
  
  def get_time_diff(datestr1, datestr2):
    last_datetime = datetime.strptime(datestr1, "%Y-%m-%d %H:%M:%S")
    current_datetime = datetime.strptime(datestr2, "%Y-%m-%d %H:%M:%S")
    time_diff = current_datetime - last_datetime
    return time_diff.total_seconds()
  
  lineList = [line.rstrip('\n') for line in open(log_filepath)]
  
  if instance == "HOROVOD_CLUSTER":
    lineList = [re.sub(r'\<[^()]*\>|\[[^()]*\]|', '', i) for i in lineList]
  
  #### Remove special charaters from horovod output
  #### Example: '2019-11-06 00:06:52 - STDOUT - INFO - [1,0]<stdout>:Epoch 3/5'
  #### Clean up to: '2019-11-06 00:06:52 - STDOUT - INFO - :Epoch 3/5'
  
  last_epoch_time = None
  last_epoch = None
  
  last_repeat_time = None
  last_repeat_ord = None
  
  for l in lineList:

    if "- REPEAT" in l:
      current_repeat_ord = l[-1:]
      current_repeat_time = l[:19]
      if last_repeat_ord is None:
        last_epoch_time = current_repeat_time
        last_repeat_ord = current_repeat_ord 
        
      elif current_repeat_ord != last_repeat_ord:
        current_epoch_time = current_repeat_time
        epoch_run_time = get_time_diff(last_epoch_time, current_epoch_time)
        
        current_repeat_ord_copy = current_repeat_ord
        current_repeat_ord = last_repeat_ord
        
        append_res()
        
        last_epoch = None
        
        current_repeat_ord = current_repeat_ord_copy
        last_repeat_ord = current_repeat_ord 
      
      
      
    if "Epoch" in l:
      current_epoch = l[-9:]
      current_epoch_time = l[:19]

      if last_epoch is None:
        epoch_run_time = get_time_diff(current_repeat_time, current_epoch_time)
        append_res()
        last_epoch = current_epoch
        last_epoch_time = current_epoch_time
        
      elif last_epoch != current_epoch:
        epoch_run_time = get_time_diff(last_epoch_time, current_epoch_time)
        ## print(f"Repeat {current_repeat_ord}/{repeat} - {last_epoch} - Start time {last_epoch_time} - End time {current_epoch_time}- {epoch_run_time} seconds")
        append_res()
        
        last_epoch_time = current_epoch_time
        last_epoch = current_epoch
        
  lastLine = lineList[-1]
  end_time = lastLine[:19]
  current_epoch_time = end_time
  
  epoch_run_time = get_time_diff(last_epoch_time, current_epoch_time)
  append_res()
#   last_epoch_time = current_epoch_time
#   last_epoch = current_epoch
      
#   last_epoch_time = None 
#   current_epoch_time = None
#   epoch_runtime
#   append_res()
  res_columns = ["STARTED_TIME", "DATASET","MODEL","TRAIN_SHAPE","TEST_SHAPE","CLUSTER", "DRIVER","WORKER","NUM_WORKERS","NP", "REPEAT", "EPOCHS", "LEARNING_RATE","BATCH_SIZE","REPEAT_ORD","EPOCH_ORD", "RUN_TIME", "FINISHED_TIME"]

  df = pd.DataFrame(res, columns=res_columns)
  
  cluster_name = df["CLUSTER"].values[0]
  np_name = df["NP"].values[0]
  df["EPOCH_ORD"].fillna("poch 0/50", inplace=True)
  df_agg = df.groupby(["CLUSTER","NP", "EPOCH_ORD"]).agg({"RUN_TIME":"mean"}).reset_index()
  df_agg["EPOCH_ORDER"] = df_agg["EPOCH_ORD"].str[-5:-3]
  df_agg["EPOCH_ORDER"] = df_agg["EPOCH_ORDER"].astype(float)
  df_agg.sort_values("EPOCH_ORDER", ascending=True, inplace=True)
  df_agg["CUM_SUM_RUN_TIME"] = df_agg["RUN_TIME"].cumsum()
  
  return cluster_name, np_name, df_agg

In [5]:
def get_benchmark_logs_summary(mapping_dict, model_name, category):
  
  category_dict = {"RUN_TIME":"RUN_TIME",
                 "CUM_RUN_TIME":"CUM_SUM_RUN_TIME"}
  
  checker = category_dict.get(category)
  if checker is None:
    raise ValueError(f"{category} is not a vlid option, please only enter from {list(category_dict.keys())} ")
    
  final_df_dict = {}
  current_mapping = mapping_dict[model_name]
  
  for np, path in current_mapping.items():
    log_paths = glob.glob(path + "*.log")
    for s_path in log_paths:
      cluster_name, np_name, df_agg = extract_result_from_log(s_path)
      final_df_dict[cluster_name+"-"+np_name] =  df_agg[category_dict.get(category)].values

  performance_df = pd.DataFrame(final_df_dict)
  performance_df.index = performance_df.index.set_names(['EPOCH_ORD'])
  performance_df = performance_df.reset_index()
  # performance_df.drop("SINGLE_INSTANCE-NA",inplace=True, axis=1)

  return performance_df
      
  

In [6]:
performance_df_inception3 = get_benchmark_logs_summary(batch_32_result_mapping, "INCEPTION3", "RUN_TIME")
performance_df_vgg16 = get_benchmark_logs_summary(batch_32_result_mapping, "VGG16", "RUN_TIME")

In [7]:
scaling_efficiency_dict = {}
scaling_efficiency_dict["VGG16"] = {}
scaling_efficiency_dict["INCEPTION3"] = {}

for i in performance_df_vgg16.columns[1:]:
  np_val = i.replace("E-","|").replace("R-","|").split("|")[1]
  scaling_efficiency = (40*6982)/performance_df_vgg16[i].sum()
  scaling_efficiency_dict["VGG16"][np_val] = scaling_efficiency
  
  
for i in performance_df_inception3.columns[1:]:
  np_val = i.replace("E-","|").replace("R-","|").split("|")[1]
  scaling_efficiency = (40*6982)/performance_df_inception3[i].sum()
  scaling_efficiency_dict["INCEPTION3"][np_val] = scaling_efficiency
  

In [8]:
import seaborn as sns

sns.set_style("whitegrid")


def get_scaling_plot_value(category):
  
  target_dict = scaling_efficiency_dict[category]
  if target_dict is None:
    raise ValueError(f"{category} is not defined!")
    
  key_list = ["NA", "1", "-8", "8", "16", "32"]
  scaling_keys = []
  scaling_values = []
  baseline_value = target_dict.get("1")
  optimal_scaling_values = []

  for key in key_list:
    if key =="NA":
      scaling_keys.append("Single")
      scaling_values.append(target_dict.get(key))
      optimal_scaling_values.append(target_dict.get(key))
    else:
      scaling_keys.append(key)
      scaling_values.append(target_dict.get(key))
      optimal_scaling_values.append(baseline_value*abs((int(key))))
      
  return scaling_keys,  scaling_values, optimal_scaling_values
      
vgg_scaling_keys,  vgg_scaling_values, vgg_optimal_scaling_values = get_scaling_plot_value("VGG16")
inc_scaling_keys,  inc_scaling_values, inc_optimal_scaling_values = get_scaling_plot_value("INCEPTION3")

vgg_scaling_efficiency = np.divide(np.asarray(vgg_scaling_values), np.asarray(vgg_optimal_scaling_values))
inc_scaling_efficiency = np.divide(np.asarray(inc_scaling_values), np.asarray(inc_optimal_scaling_values))

import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 2, sharey=True, figsize=(12,5))

sns.barplot(vgg_scaling_keys, vgg_scaling_values, color='royalblue', ax=axes[0])
sns.barplot(vgg_scaling_keys, vgg_optimal_scaling_values, color='royalblue', facecolor=(1, 1, 1, 0), errcolor=".2", edgecolor=".2", ax=axes[0])

num = 0
for p in axes[0].patches[6:]:
  if num < 2:
    num += 1
    continue
  percentage = '{:.1f}%'.format(100 * vgg_scaling_efficiency[num])
  x = p.get_x() + p.get_width()/6
  y = p.get_y() + p.get_height() + 0.05
  axes[0].annotate(percentage, (x, y))
  num += 1
  

axes[0].set(xlabel='VGG-16', ylabel='Images/Sec')


sns.barplot(inc_scaling_keys, inc_scaling_values, color='royalblue', ax=axes[1])
sns.barplot(inc_scaling_keys, inc_optimal_scaling_values, color='royalblue', facecolor=(1, 1, 1, 0), errcolor=".2", edgecolor=".2", ax=axes[1])

num = 0
for p in axes[1].patches[6:]:
  if num < 2:
    num += 1
    continue
  percentage = '{:.1f}%'.format(100 * inc_scaling_efficiency[num])
  x = p.get_x() + p.get_width()/6
  y = p.get_y() + p.get_height() + 0.05
  axes[1].annotate(percentage, (x, y))
  num += 1


  
axes[1].set(xlabel='Inception V3')

fig.suptitle('Training with Caltech101 Data on P2.8xlarge Instances')
plt.subplots_adjust(wspace=0)

display()



In [9]:
from scipy import stats
performance_df_vgg16.describe()

,EPOCH_ORD,SINGLE_INSTANCE-NA,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-8,HOROVOD_CLUSTER-16,HOROVOD_CLUSTER-32
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,20.000000,237.707317,71.560976,276.707317,70.585366,61.560976,46.829268
std,11.979149,47.094715,12.857000,62.545681,11.960300,12.520081,12.846989
min,0.000000,31.000000,10.000000,43.000000,16.000000,50.000000,38.000000
25%,10.000000,237.000000,68.000000,271.000000,68.000000,54.000000,41.000000
50%,20.000000,237.000000,71.000000,273.000000,70.000000,57.000000,42.000000
75%,30.000000,238.000000,75.000000,277.000000,72.000000,59.000000,46.000000
max,40.000000,452.000000,105.000000,594.000000,104.000000,100.000000,104.000000


In [10]:
stats.f_oneway(performance_df_vgg16["SINGLE_INSTANCE-NA"],
               performance_df_vgg16["HOROVOD_CLUSTER-1"],
               performance_df_vgg16["HOROVOD_CLUSTER--8"],
              performance_df_vgg16["HOROVOD_CLUSTER-8"],
              performance_df_vgg16["HOROVOD_CLUSTER-16"],
              performance_df_vgg16["HOROVOD_CLUSTER-32"])

### 375.787, d.f(5,234) p= 2.603e-111

Out[10]: F_onewayResult(statistic=375.7866214558725, pvalue=2.603407364796959e-111)

In [11]:
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
from itertools import repeat

result_data = []
for col in performance_df_vgg16.columns[1:]:
  temp_df = pd.DataFrame(list(zip(performance_df_vgg16[col],repeat(col))))
  result_data.append(temp_df)
vgg16_pairwise_tukey_df = pd.concat(result_data)
vgg16_pairwise_tukey_df.columns = ["RUN_TIME","CLUSTER"]
vgg16_res = pairwise_tukeyhsd(vgg16_pairwise_tukey_df["RUN_TIME"], vgg16_pairwise_tukey_df["CLUSTER"], 0.05)

In [12]:
vgg16_res_table = vgg16_res._results_table
vgg16_res_list = []
col_name_list = vgg16_res_table.data[0]
col_name_list.append("p_values")

from statsmodels.stats.libqsturng import psturng
pvalue_lists = psturng(np.abs(vgg16_res.meandiffs / vgg16_res.std_pairs), len(vgg16_res.groupsunique), vgg16_res.df_total)

for _row in range(0, len( vgg16_res_table.data)-1):
  pvalues = pvalue_lists[_row]
  _row += 1
  temp_list = vgg16_res_table.data[_row]
  temp_list.append(pvalues)
  vgg16_res_list.append(temp_list)
pd.DataFrame(vgg16_res_list, columns=col_name_list )

,group1,group2,meandiff,lower,upper,reject,p_values
0,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-1,205.1463,183.8483,226.4444,True,0.001000
1,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-16,-10.0000,-31.2980,11.2980,False,0.729855
2,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-32,-24.7317,-46.0297,-3.4337,True,0.012492
3,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-8,-0.9756,-22.2736,20.3224,False,0.900000
4,HOROVOD_CLUSTER--8,SINGLE_INSTANCE-NA,166.1463,144.8483,187.4444,True,0.001000
5,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-16,-215.1463,-236.4444,-193.8483,True,0.001000
6,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-32,-229.8780,-251.1761,-208.5800,True,0.001000
7,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-8,-206.1220,-227.4200,-184.8239,True,0.001000
8,HOROVOD_CLUSTER-1,SINGLE_INSTANCE-NA,-39.0000,-60.2980,-17.7020,True,0.001000
9,HOROVOD_CLUSTER-16,HOROVOD_CLUSTER-32,-14.7317,-36.0297,6.5663,False,0.353226


In [13]:
scaling_efficiency_dict["VGG16"]

Out[13]: {'NA': 28.655858813872356,
 '-8': 95.18745739604636,
 '1': 24.617011899515205,
 '8': 96.50310988251555,
 '16': 110.64976228209191,
 '32': 145.45833333333334}

In [15]:
from scipy import stats
performance_df_inception3.describe()

,EPOCH_ORD,SINGLE_INSTANCE-NA,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-8,HOROVOD_CLUSTER-16,HOROVOD_CLUSTER-32
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,20.000000,315.658537,50.609756,316.292683,49.609756,31.390244,20.195122
std,11.979149,72.593598,7.962029,69.484618,6.689088,7.895816,7.149194
min,0.000000,18.000000,23.000000,27.000000,23.000000,24.000000,16.000000
25%,10.000000,313.000000,49.000000,312.000000,48.000000,28.000000,17.000000
50%,20.000000,314.000000,49.000000,315.000000,49.000000,29.000000,18.000000
75%,30.000000,315.000000,51.000000,317.000000,51.000000,32.000000,20.000000
max,40.000000,665.000000,86.000000,646.000000,80.000000,66.000000,54.000000


In [16]:
stats.f_oneway(performance_df_inception3["SINGLE_INSTANCE-NA"],
               performance_df_inception3["HOROVOD_CLUSTER-1"],
               performance_df_inception3["HOROVOD_CLUSTER--8"],
              performance_df_inception3["HOROVOD_CLUSTER-8"],
              performance_df_inception3["HOROVOD_CLUSTER-16"],
              performance_df_inception3["HOROVOD_CLUSTER-32"])

### 494.492, d.f(5,234) p= 3.66248e-124

Out[15]: F_onewayResult(statistic=494.4922841700079, pvalue=3.6624874957286216e-124)

In [17]:
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
from itertools import repeat

result_data = []
for col in performance_df_inception3.columns[1:]:
  temp_df = pd.DataFrame(list(zip(performance_df_inception3[col],repeat(col))))
  result_data.append(temp_df)
inception3_pairwise_tukey_df = pd.concat(result_data)
inception3_pairwise_tukey_df.columns = ["RUN_TIME","CLUSTER"]
inception3_res = pairwise_tukeyhsd(inception3_pairwise_tukey_df["RUN_TIME"], inception3_pairwise_tukey_df["CLUSTER"], 0.05)

In [18]:
inception3_res_table = inception3_res._results_table
inception3_res_list = []
col_name_list = inception3_res_table.data[0]
col_name_list.append("p_values")

from statsmodels.stats.libqsturng import psturng
pvalue_lists = psturng(np.abs(inception3_res.meandiffs / inception3_res.std_pairs), len(inception3_res.groupsunique), inception3_res.df_total)

for _row in range(0, len( inception3_res_table.data)-1):
  pvalues = pvalue_lists[_row]
  _row += 1
  temp_list = inception3_res_table.data[_row]
  temp_list.append(pvalues)
  inception3_res_list.append(temp_list)
pd.DataFrame(inception3_res_list, columns=col_name_list )

,group1,group2,meandiff,lower,upper,reject,p_values
0,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-1,265.6829,239.3684,291.9974,True,0.001000
1,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-16,-19.2195,-45.5340,7.0950,False,0.291764
2,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-32,-30.4146,-56.7291,-4.1001,True,0.013134
3,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-8,-1.0000,-27.3145,25.3145,False,0.900000
4,HOROVOD_CLUSTER--8,SINGLE_INSTANCE-NA,265.0488,238.7343,291.3633,True,0.001000
5,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-16,-284.9024,-311.2169,-258.5879,True,0.001000
6,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-32,-296.0976,-322.4121,-269.7831,True,0.001000
7,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-8,-266.6829,-292.9974,-240.3684,True,0.001000
8,HOROVOD_CLUSTER-1,SINGLE_INSTANCE-NA,-0.6341,-26.9486,25.6803,False,0.900000
9,HOROVOD_CLUSTER-16,HOROVOD_CLUSTER-32,-11.1951,-37.5096,15.1194,False,0.802550


In [19]:
scaling_efficiency_dict["INCEPTION3"]

Out[18]: {'NA': 21.579354041106477,
 '-8': 134.59277108433736,
 '1': 21.536088834053054,
 '8': 137.30580137659783,
 '16': 217.000777000777,
 '32': 337.29468599033817}

In [20]:
%sh ls /dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_13_23_12_23

benchmark|PROD|MNIST|CNN|60000|10000|HOROVOD_CLUSTER|c4.2xlarge|c4.2xlarge|8|16|3|50|0.1|128.log
benchmark|PROD|MNIST|CNN|60000|10000|HOROVOD_CLUSTER|c4.2xlarge|c4.2xlarge|8|32|3|50|0.1|128.log
benchmark|PROD|MNIST|CNN|60000|10000|HOROVOD_CLUSTER|c4.2xlarge|c4.2xlarge|8|8|3|50|0.1|128.log
model

In [21]:
cnn_result_mapping = {"CNN":
                                {"single": '/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_13_11_09_19/',
                                  "-8124": '/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_13_14_31_27/',
                                 "8_16_32": '/dbfs/mnt/wendao_test2/horovod_benchmark/logs/2019_11_13_23_12_23/',
                                }
                     }

In [22]:
def get_benchmark_logs_summary(mapping_dict, model_name, category):
  
  category_dict = {"RUN_TIME":"RUN_TIME",
                 "CUM_RUN_TIME":"CUM_SUM_RUN_TIME"}
  
  checker = category_dict.get(category)
  if checker is None:
    raise ValueError(f"{category} is not a vlid option, please only enter from {list(category_dict.keys())} ")
    
  final_df_dict = {}
  current_mapping = mapping_dict[model_name]
  
  for np, path in current_mapping.items():
    log_paths = glob.glob(path + "*.log")
    for s_path in log_paths:
      cluster_name, np_name, df_agg = extract_result_from_log(s_path)
      final_df_dict[cluster_name+"-"+np_name] =  df_agg[category_dict.get(category)].values

  performance_df = pd.DataFrame(final_df_dict)
  performance_df.index = performance_df.index.set_names(['EPOCH_ORD'])
  performance_df = performance_df.reset_index()
  # performance_df.drop("SINGLE_INSTANCE-NA",inplace=True, axis=1)

  return performance_df
      
  

In [23]:
performance_df_cnn = get_benchmark_logs_summary(cnn_result_mapping, "CNN", "RUN_TIME")
performance_df_cnn_plot = get_benchmark_logs_summary(cnn_result_mapping, "CNN", "CUM_RUN_TIME")

In [24]:
scaling_efficiency_dict = {}
scaling_efficiency_dict["CNN"] = {}

for i in performance_df_cnn.columns[1:]:
  np_val = i.replace("E-","|").replace("R-","|").split("|")[1]
  scaling_efficiency = (40*6982)/performance_df_cnn[i].sum()
  scaling_efficiency_dict["CNN"][np_val] = scaling_efficiency
  

In [25]:
from scipy import stats
performance_df_cnn.describe()

,EPOCH_ORD,SINGLE_INSTANCE-NA,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-2,HOROVOD_CLUSTER-4,HOROVOD_CLUSTER-8,HOROVOD_CLUSTER-16,HOROVOD_CLUSTER-32
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,25.000000,29.771242,37.130719,31.980392,21.843137,13.477124,9.366013,7.849673,7.653595
std,14.866069,4.123370,4.318476,2.542450,1.645672,1.217018,1.714325,1.172488,0.997688
min,0.000000,1.000000,8.333333,15.333333,11.000000,9.000000,8.000000,6.666667,7.000000
25%,12.500000,30.166667,37.000000,31.833333,21.666667,13.000000,8.500000,7.333333,7.333333
50%,25.000000,30.333333,37.333333,32.000000,22.000000,13.333333,9.000000,7.666667,7.333333
75%,37.500000,30.500000,38.000000,32.333333,22.166667,13.666667,9.333333,7.833333,7.666667
max,50.000000,31.666667,43.666667,37.000000,24.000000,19.000000,16.000000,14.000000,13.000000


In [26]:
performance_df_cnn.columns

Out[25]: Index(['EPOCH_ORD', 'SINGLE_INSTANCE-NA', 'HOROVOD_CLUSTER--8',
 'HOROVOD_CLUSTER-1', 'HOROVOD_CLUSTER-2', 'HOROVOD_CLUSTER-4',
 'HOROVOD_CLUSTER-8', 'HOROVOD_CLUSTER-16', 'HOROVOD_CLUSTER-32'],
 dtype='object')

In [27]:
stats.f_oneway(performance_df_cnn["SINGLE_INSTANCE-NA"],
               performance_df_cnn["HOROVOD_CLUSTER-1"],
               performance_df_cnn["HOROVOD_CLUSTER--8"],
              performance_df_cnn["HOROVOD_CLUSTER-2"],
              performance_df_cnn["HOROVOD_CLUSTER-4"],
              performance_df_cnn["HOROVOD_CLUSTER-8"],
              performance_df_cnn["HOROVOD_CLUSTER-16"],
              performance_df_cnn["HOROVOD_CLUSTER-32"]
              )

### 1119.61, d.f(5,234) p= 2.603e-111

Out[26]: F_onewayResult(statistic=1119.6116002308813, pvalue=2.7663124059999763e-258)

In [28]:
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
from itertools import repeat

result_data = []
for col in performance_df_cnn.columns[1:]:
  temp_df = pd.DataFrame(list(zip(performance_df_cnn[col],repeat(col))))
  result_data.append(temp_df)
cnn_pairwise_tukey_df = pd.concat(result_data)
cnn_pairwise_tukey_df.columns = ["RUN_TIME","CLUSTER"]
cnn_res = pairwise_tukeyhsd(cnn_pairwise_tukey_df["RUN_TIME"], cnn_pairwise_tukey_df["CLUSTER"], 0.05)

In [29]:
cnn_res_table = cnn_res._results_table
cnn_res_list = []
col_name_list = cnn_res_table.data[0]
col_name_list.append("p_values")

from statsmodels.stats.libqsturng import psturng
pvalue_lists = psturng(np.abs(cnn_res.meandiffs / cnn_res.std_pairs), len(cnn_res.groupsunique), cnn_res.df_total)

for _row in range(0, len( cnn_res_table.data)-1):
  pvalues = pvalue_lists[_row]
  _row += 1
  temp_list = cnn_res_table.data[_row]
  temp_list.append(pvalues)
  cnn_res_list.append(temp_list)
cnn_tukey_df = pd.DataFrame(cnn_res_list, columns=col_name_list )
cnn_tukey_df[(cnn_tukey_df["group1"]=='HOROVOD_CLUSTER-1')|(cnn_tukey_df["group2"]=='HOROVOD_CLUSTER-1')]

,group1,group2,meandiff,lower,upper,reject,p_values
0,HOROVOD_CLUSTER--8,HOROVOD_CLUSTER-1,-5.1503,-6.6830,-3.6177,True,0.001
7,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-16,-24.1307,-25.6634,-22.5981,True,0.001
8,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-2,-10.1373,-11.6699,-8.6046,True,0.001
9,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-32,-24.3268,-25.8594,-22.7941,True,0.001
10,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-4,-18.5033,-20.0359,-16.9706,True,0.001
11,HOROVOD_CLUSTER-1,HOROVOD_CLUSTER-8,-22.6144,-24.1470,-21.0817,True,0.001
12,HOROVOD_CLUSTER-1,SINGLE_INSTANCE-NA,-2.2092,-3.7418,-0.6765,True,0.001


In [30]:
scaling_efficiency_dict["CNN"]

Out[29]: {'NA': 183.93852908891327,
 '-8': 147.48107727512763,
 '1': 171.23237277743715,
 '2': 250.70017953321366,
 '4': 406.3239573229873,
 '8': 584.6755059316121,
 '16': 697.6186511240633,
 '32': 715.4910333048676}

In [31]:
performance_df_cnn_plot = pd.melt(performance_df_cnn_plot, ['EPOCH_ORD'])
performance_df_cnn_plot.rename(columns={'EPOCH_ORD':'Epoches',"variable":"Cluster Config","value":"Cumulative Runtime"}, inplace=True)

In [32]:
sns.set(rc={'figure.figsize':(20,10), "lines.linewidth": 3.5, },font_scale=1.5 )
sns.set_style('ticks')

fig, ax = plt.subplots()

ax = sns.lineplot(x='Epoches', y='Cumulative Runtime', hue='Cluster Config', 
             data=performance_df_cnn_plot)

display(fig)



In [33]:
%sh mkdir /dbfs/mnt/wendao_test2/horovod_benchmark/data

In [34]:
performance_df_cnn.to_csv("/dbfs/mnt/wendao_test2/horovod_benchmark/data/cnn_mnist_benchmark.csv", index=False)
performance_df_inception3.to_csv("/dbfs/mnt/wendao_test2/horovod_benchmark/data/inception-V3_benchmark.csv", index=False)
performance_df_vgg16.to_csv("/dbfs/mnt/wendao_test2/horovod_benchmark/data/vgg16_benchmark.csv", index=False)